In [1]:
from glob import glob
from tqdm import tqdm
import json
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random

In [2]:
extra = [
 '/home/ubuntu/dedup-text-dataset/gov.my.jsonl',
 '/home/ubuntu/dedup-text-dataset/edu.my.jsonl'
]
# for f in extra:
#     f = os.path.split(f)[1]
#     url = f'https://huggingface.co/datasets/malaysia-ai/online-articles-partition/resolve/main/{f}.tokenized'
#     os.system(f'wget {url}')

In [3]:
files = []

for f in extra:
    f = os.path.split(f)[1]
    files.append(f'{f}.tokenized')
    
len(files)

2

In [4]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from huggingface_hub import InferenceClient

generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=4096,
    top_p=0.95,
    top_k=50,
    repetition_penalty=1.0,
    do_sample=True,
)

client = InferenceClient(
    'https://mixtral.us-west-2.mesolitica.com/', timeout = 50
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

def format_user(history):
    prompt = "<s>"
    for user_prompt, bot_response in history:
        prompt += f"[INST] {user_prompt} [/INST]"
        prompt += f" {bot_response}</s> "
    prompt += f"[INST]"
    return prompt

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [5]:
partitions = []
total = 0
for f in tqdm(files):
    total = 0
    filename = os.path.split(f)[1].replace('.tokenized', '')
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            partitions.append((l, filename))
            total += 1
    
len(partitions)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.21s/it]


198955

In [6]:
with open('random-reply-user.json') as fopen:
    random_user = json.load(fopen)
    
with open('stupid-random-reply-user.json') as fopen:
    stupid_random_user = json.load(fopen)
    
disagree = [
    'hi', 'hi hi', 'saya nak tanya soalan boleh', 'hello'
    'bullshit', 'u are lying', 'tipu la ko', 'penipu', 'tipu lah sial', 'ko tipu', 'aku tak caye',
    'wtf tipu', 'ye ke ko ni, tipu lah', 'tipu sial', 'bodoh salah', 'tak betul la sial',
    'tak betul', 'salah', 'ko pasti ke ni', 'tipu tipu tipu',
]

with open('disagree.json') as fopen:
    disagree.extend(json.load(fopen)[:1000])
    
with open('doubt.json') as fopen:
    disagree.extend(json.load(fopen)[:1000])
    
disagree = [d for d in disagree if len(d) > 2]
len(disagree)

2018

In [7]:
with open('malay-instructions.json') as fopen:
    malay_instructions = json.load(fopen)
    
len(malay_instructions)

605702

In [8]:
!mkdir rag-multiturn-chaotic-part2

mkdir: cannot create directory ‘rag-multiturn-chaotic-part2’: File exists


In [9]:
# q = partitions[0]

In [10]:
# prompt = f'{q[0]}\n\ngenerate questions based on context above'
# formatted_prompt = format_prompt(prompt, [])

# while True:
#     try:
#         stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
#         output = stream.generated_text
#         break
#     except:
#         pass

# splitted = output.strip().split('\n')
# splitted = [s for s in splitted if '.' if s and '?' in s]
# splitted = ['.'.join(s.split('.')[1:]).strip() for s in splitted]
# splitted = [s for s in splitted if len(s) > 3]
# splitted = [s[1:] if s[0] == '"' else s for s in splitted]
# questions = [s[:-1] if s[-1] == '"' else s for s in splitted]
# questions = [s for s in questions if len(s) > 20]

In [11]:
# history = []
# for _ in range(random.randint(2, 5)):
#     choice = random.randint(0, 5)
#     if choice in [0,1]:
#         rq = random.choice(random_user)
#     elif choice in [2]:
#         rq = random.choice(disagree)
#     elif choice in [3]:
#         rq = random.choice(stupid_random_user)
#     else:
#         rq = random.choice(questions)
#     u = f"""
# knowledge base is below.
# ---------------------
# {q[0]}
# ---------------------
# Given the knowledge base and not prior knowledge, respond user question.
# Question: {rq}
# """
#     formatted_prompt = format_prompt(u, history)
#     while True:
#         try:
#             stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
#             a = stream.generated_text.strip()
#             break
#         except Exception as e:
#             # print(e)
#             pass

#     history.append((rq, a))
    
# history

In [12]:
def answer(q, i):
    filename = f'rag-multiturn-chaotic-part2/{i}.json'
    if os.path.exists(filename):
        return
    
    prompt = f'{q[0]}\n\ngenerate questions based on context above'
    formatted_prompt = format_prompt(prompt, [])
    
    while True:
        try:
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            break
        except:
            pass
    
    splitted = output.strip().split('\n')
    splitted = [s for s in splitted if '.' if s and '?' in s]
    splitted = ['.'.join(s.split('.')[1:]).strip() for s in splitted]
    splitted = [s for s in splitted if len(s) > 3]
    splitted = [s[1:] if s[0] == '"' else s for s in splitted]
    questions = [s[:-1] if s[-1] == '"' else s for s in splitted]
    questions = [s for s in questions if len(s) > 20]
    
    if not len(questions):
        with open(filename, 'w') as fopen:
            json.dump(False, fopen)
        return
    
    history = []
    questions_set = set(questions)
    selected_q = set()
    for _ in range(random.randint(2, 5)):
        if len(list(questions_set - selected_q)) == 0:
            break
        choice = random.randint(0, 10)
        if choice in [0]:
            rq = random.choice(random_user)
        elif choice in [1]:
            rq = random.choice(disagree)
        elif choice in [2]:
            rq = random.choice(stupid_random_user)
        elif choice in [3]:
            rq = random.choice(malay_instructions)
        else:
            rq = random.choice(list(questions_set - selected_q))
            selected_q.add(rq)
            
        u = f"""
knowledge base is below.
---------------------
{q[0]}
---------------------
Given the knowledge base and not prior knowledge, respond user question.
Question: {rq}
"""
        formatted_prompt = format_prompt(u.strip(), history)
        while True:
            try:
                stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
                a = stream.generated_text.strip()
                break
            except Exception as e:
                # print(e)
                pass

        history.append((rq, a))
    
    d = {
        'filename': q[1],
        'context': q[0],
        'questions': questions,
        'multiturn': history
    }
    
    with open(filename, 'w') as fopen:
        json.dump(d, fopen)

In [13]:
answer(partitions[1], 1)

In [14]:
with open('rag-multiturn-chaotic-part2/1.json') as fopen:
    d = json.load(fopen)

In [15]:
d['multiturn']

[['What form should be used for the application of the Tambang Mengunjungi Wilayah Asal?',
  'Based on the knowledge base provided, the form to be used for the application of the Tambang Mengunjungi Wilayah Asal is Lampiran A, Pekeliling Perkhidmatan ini. It is mentioned in point m of the knowledge base that the application for using the facility should be made using the form in Lampiran A.'],
 ['And how much does a duck weigh?',
  'I\'m sorry for any confusion, but the provided knowledge base does not contain any information about the weight of a duck. The knowledge base consists of a policy regarding the use of a facility called "Tambang Mengunjungi Wilayah Asal" by certain government employees, as well as instructions for using a web-based dashboard for strategic planning. If you have any questions related to this topic, I would be happy to try to answer them.']]

In [16]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [17]:
from threading import Thread
from queue import Queue

queue = Queue()
urls = [(q, no) for no, q in enumerate(partitions)]
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [ ]:
max_worker = 600
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

 87%|███████████████████████████████████████████████████████████████████████████▋           | 173124/198955 [12:12:36<3:49:28,  1.88it/s]

In [2]:
files = glob('rag-multiturn-chaotic-part2/*.json')

In [3]:
all_texts = set()
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    if not data:
        continue
    for d in data['multiturn']:
        all_texts |= set(d)
len(all_texts)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 172542/172542 [00:33<00:00, 5211.98it/s]


1076592

In [4]:
with open('rag-multiturn-chaotic-part2.texts', 'w') as fopen:
    for t in all_texts:
        if not len(t):
            continue
        fopen.write(f'{json.dumps(t)}\n')

In [5]:
!cp rag-multiturn-chaotic-part2.texts ../translation

In [6]:
import re

mapping = {}
for f in glob('/home/husein/ssd3/translation/rag-multiturn-chaotic-part2.texts*.splitted.requested'):
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if 'Source text\nclear\nLook up details' in l['r']['result']:
                continue
            if 'clear\nLook up details' in l['r']['result']:
                continue
            if l['r']['result'].startswith('Source text\n'):
                continue

            n = l['r']['result']
            hypens = re.findall('\w+ -\w+', n)
            for h in hypens:
                splitted = h.split('-')
                if len(splitted) != 2:
                    continue
                splitted = [s.strip() for s in splitted]
                splitted = '-'.join(splitted)
                n = n.replace(h, splitted)
            mapping[l['src']] = n
            
len(mapping)

1076533

In [15]:
with open('rag-multiturn-chaotic-part2.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
            
        if not data:
            continue
            
        chat = []
        for d in data['multiturn']:
            chat.extend([
                {'role': 'user', 'content': d[0], 'content_ms': mapping.get(d[0])},
                {'role': 'assistant', 'content': d[1], 'content_ms': mapping.get(d[1])}
            ])

        d = {
            'filename': data['filename'],
            'chat': chat,
            'context': data['context'],
            'questions': data['questions']
        }
        fopen_l.write(f'{json.dumps(d)}\n')

100%|██████████████████████████████████████████████████████████████████████████████████████████| 172542/172542 [00:49<00:00, 3512.13it/s]


In [16]:
!head -n 1 rag-multiturn-chaotic-part2.jsonl

{"filename": "gov.my.jsonl", "chat": [{"role": "user", "content": "Are you as sure as a horse's hind leg?", "content_ms": "Adakah anda pasti seperti kaki belakang kuda?"}, {"role": "assistant", "content": "I'm afraid I don't understand the reference to a horse's hind leg in relation to my ability to provide accurate information. However, I can assure you that I will provide information based solely on the knowledge base you've given me and to the best of my ability, I will provide accurate and relevant responses. I don't have feelings, beliefs, or personal knowledge, I am a machine learning model that generate text based on the input data.", "content_ms": "Saya takut saya tidak memahami rujukan kepada kaki belakang kuda berkaitan dengan keupayaan saya untuk memberikan maklumat yang tepat. Walau bagaimanapun, saya boleh memberi jaminan kepada anda bahawa saya akan memberikan maklumat hanya berdasarkan pangkalan pengetahuan yang anda berikan kepada saya dan setakat kemampuan saya, saya a

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='rag-multiturn-chaotic-part2.jsonl',
    path_in_repo='mrag-multiturn-chaotic-part2.jsonl',
    repo_id='mesolitica/mixtral-malaysian-rag',
    repo_type='dataset',
)

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


rag-multiturn-chaotic-part2.jsonl:   0%|          | 0.00/2.45G [00:00<?, ?B/s]